In [2]:
%%bash 
pushd /media/Data1/Data_Analysis/

qiime tools import \
--type 'SampleData[PairedEndSequencesWithQuality]' \
--input-path sample_data.tsv \
--output-path paired-end-demux.qza \
--input-format PairedEndFastqManifestPhred33V2

popd

/media/Data1/Data_Analysis ~
Imported sample_data.tsv as PairedEndFastqManifestPhred33V2 to paired-end-demux.qza
~


In [3]:
%%bash 
pushd /media/Data1/Data_Analysis/

qiime cutadapt trim-paired \
--i-demultiplexed-sequences paired-end-demux.qza \
--p-front-f CCTACGGGAGGCTGCAG \
--p-front-r GACTACAGGGGTATCTAATCC \
--o-trimmed-sequences trimmed-paired-end-demux.qza

popd

/media/Data1/Data_Analysis ~
Saved SampleData[PairedEndSequencesWithQuality] to: trimmed-paired-end-demux.qza
~


In [4]:
%%bash 
pushd /media/Data1/Data_Analysis/

qiime demux summarize \
--i-data trimmed-paired-end-demux.qza \
--o-visualization demux.qzv

popd

/media/Data1/Data_Analysis ~
Saved Visualization to: demux.qzv
~


In [ ]:
Przycinanie + ocena

In:
    - trimmed-paired-end-demux.qza
Out:
    - rep-seqs.qza # representative sequences 
    - table.qza
    - dadastats.qza

In:
    - dadastats.qza
Out:
    - denoising-stats.qzv

In:
    - sample_metadata.tsv
    - table.qza
Out:
    - table.qzv

In:
    - rep-seqs.qza
Out:
    - rep-seqs.qzv

In [14]:
%%bash 
pushd /media/Data1/Data_Analysis/

qiime dada2 denoise-paired \
--i-demultiplexed-seqs trimmed-paired-end-demux.qza \
--p-trim-left-f 0 \
--p-trim-left-r 0 \
--p-trunc-len-f 270 \
--p-trunc-len-r 170 \
--p-n-threads 12 \
--o-representative-sequences rep-seqs.qza \
--o-table table.qza \
--o-denoising-stats dadastats.qza

qiime metadata tabulate \
  --m-input-file dadastats.qza \
  --o-visualization denoising-stats.qzv
  
qiime feature-table summarize \
  --i-table table.qza \
  --o-visualization table.qzv \
  --m-sample-metadata-file sample_metadata.tsv

qiime feature-table tabulate-seqs \
  --i-data rep-seqs.qza \
  --o-visualization rep-seqs.qzv

popd

/media/Data1/Data_Analysis ~
Saved FeatureTable[Frequency] to: table.qza
Saved FeatureData[Sequence] to: rep-seqs.qza
Saved SampleData[DADA2Stats] to: dadastats.qza
Saved Visualization to: denoising-stats.qzv
Saved Visualization to: table.qzv
Saved Visualization to: rep-seqs.qzv
~


In [ ]:
#qiime tools export --input-path Analysis/denoising-stats.qza --output-path Analysis/dada2_output

In [ ]:
SILVA zamist greegenes

In [9]:
%%bash
pushd /media/Data1/Data_Analysis/

qiime tools import \
 --type FeatureData[Sequence] \
 --input-path $HOME/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna \
 --output-path 99_otus_16S

qiime tools import \
--type FeatureData[Taxonomy] \
--input-path $HOME/SILVA_132_QIIME_release/taxonomy/16S_only/99/majority_taxonomy_7_levels.txt \
--input-format HeaderlessTSVTaxonomyFormat \
--output-path majority_taxonomy_7_levels

popd

/media/Data1/Data_Analysis ~
Imported /home/amnesia/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/99/silva_132_99_16S.fna as DNASequencesDirectoryFormat to 99_otus_16S
Imported /home/amnesia/SILVA_132_QIIME_release/taxonomy/16S_only/99/majority_taxonomy_7_levels.txt as HeaderlessTSVTaxonomyFormat to majority_taxonomy_7_levels
~


In [15]:
%%bash
pushd /media/Data1/Data_Analysis/

qiime feature-classifier classify-consensus-blast \
--i-query rep-seqs.qza \
--i-reference-taxonomy majority_taxonomy_7_levels.qza \
--i-reference-reads 99_otus_16S.qza \
--o-classification taxonomy \
--p-perc-identity 0.90 \
--p-maxaccepts 1

popd

/media/Data1/Data_Analysis ~
Saved FeatureData[Taxonomy] to: taxonomy.qza
~


In [16]:
%%bash
pushd /media/Data1/Data_Analysis/

qiime diversity alpha-rarefaction \
  --i-table table.qza \
  --i-phylogeny rooted-tree.qza \
  --p-max-depth 23000 \
  --m-metadata-file sample_metadata.tsv \
  --o-visualization alpha-rarefaction.qzv

popd

/media/Data1/Data_Analysis ~
Saved Visualization to: alpha-rarefaction.qzv
~


In [17]:
%%bash
pushd /media/Data1/Data_Analysis/

qiime diversity beta-rarefaction \
--i-table table.qza \
--p-metric weighted_unifrac \
--p-clustering-method nj \
--m-metadata-file sample_metadata.tsv \
--p-sampling-depth 23000 \
--i-phylogeny rooted-tree.qza \
--o-visualization weighted-unifrac.qzv

popd

/media/Data1/Data_Analysis ~
Saved Visualization to: weighted-unifrac.qzv
~


In [12]:
%%bash 
pushd /media/Data1/Data_Analysis/

qiime phylogeny align-to-tree-mafft-fasttree \
  --i-sequences rep-seqs.qza \
  --o-alignment aligned-rep-seqs.qza \
  --o-masked-alignment masked-aligned-rep-seqs.qza \
  --o-tree unrooted-tree.qza \
  --o-rooted-tree rooted-tree.qza
  
popd

/media/Data1/Data_Analysis ~
Saved FeatureData[AlignedSequence] to: aligned-rep-seqs.qza
Saved FeatureData[AlignedSequence] to: masked-aligned-rep-seqs.qza
Saved Phylogeny[Unrooted] to: unrooted-tree.qza
Saved Phylogeny[Rooted] to: rooted-tree.qza
~


In [7]:
qza_file = 'dadastats.qza'
metadata_file = 'sample_metadata.tsv'

dokdo.denoising_stats_plot(
    qza_file,
    metadata_file,
    'group',
    figsize=(8, 6)
)

plt.tight_layout()

AttributeError: module 'dokdo.dokdo' has no attribute 'denoising_stats_plot'